# 196 Preliminary Work

In this notebook, we demonstrate and implement a prequential pipeline consisting of an Adaptive Random Forest (ARF) online classifier, ADWIN2 concept drift detector, and both OFS and FIRES online feature extraction and selection for comparison.

At this point, all modules used are in their default parameters.

Note: Install dependencies first via pip install -r requirements.txt

### Imports

In [ ]:
# Main Imports
from skmultiflow.meta import AdaptiveRandomForestClassifier # ARF
from skmultiflow.lazy import KNNClassifier, KNNADWINClassifier # KNN
from sklearn.svm import LinearSVC # SVM

from skmultiflow.evaluation import EvaluatePrequential # Pipeline
from skmultiflow.drift_detection import ADWIN # ADWIN drift detection
from float.feature_selection import OFS, FIRES # online feature methods

# Other imports
import matplotlib.pyplot as plt